# 06. Agent Evaluation (Agent 평가)

이 노트북에서는 배포된 Agent의 품질과 성능을 평가합니다.

## 목표

- Azure AI Evaluation SDK를 사용한 Agent 평가
- 성능, 정확도, 안전성 메트릭 수집
- 평가 결과 분석 및 시각화

## 사전 요구사항

1. Notebook 01-03 완료 (Azure 리소스 및 Agent 배포)
2. `config.json` 파일 존재
3. Azure AI Project 접근 권한

---

## 1. 환경 설정 및 Config 로드

이전 노트북에서 생성한 설정을 로드합니다.

In [ ]:
import json
import sys
import subprocess
from pathlib import Path

# config.json 로드
config_path = Path("config.json")
if not config_path.exists():
    raise FileNotFoundError("config.json not found. Please run notebooks 01-03 first.")

with open(config_path, 'r') as f:
    config = json.load(f)

# 주요 변수 추출
PROJECT_CONNECTION_STRING = config.get("project_connection_string", "")
simple_project_conn = PROJECT_CONNECTION_STRING.split(';')[0] if PROJECT_CONNECTION_STRING else ""

print("=== Configuration Loaded ===")
print(f"Resource Group: {config.get('resource_group')}")
print(f"Location: {config.get('location')}")
print(f"Project: {simple_project_conn}")
print(f"Model: {config.get('model_deployment_name')}")
print("\n" + "="*50)

## 2. Azure AI Evaluation 패키지 설치

Agent 평가에 필요한 패키지를 설치합니다.

In [ ]:
# Azure AI Evaluation 패키지 설치
print("=== Installing Azure AI Evaluation Package ===\n")

result = subprocess.run(
    [sys.executable, "-m", "pip", "install", "-q", "azure-ai-evaluation"],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("✅ azure-ai-evaluation installed successfully")
else:
    print(f"⚠️  Installation warning: {result.stderr}")
    print("   Proceeding anyway...")

print("\n" + "="*50)

## 3. Evaluation 개요

### Evaluation 유형

**Performance Evaluators (성능 평가):**
- **Intent Resolution**: Agent가 사용자 의도를 올바르게 파악했는지 평가
- **Tool Call Accuracy**: Agent가 올바른 도구를 정확하게 호출했는지 평가  
- **Task Adherence**: Agent가 지시사항을 충실히 따랐는지 평가

**Safety Evaluators (안전성 평가):**
- **Content Safety**: 부적절한 콘텐츠(폭력, 혐오 등) 포함 여부 평가
- **Indirect Attack**: 간접적인 악의적 공격 시도 감지
- **Code Vulnerability**: 생성된 코드의 보안 취약점 평가

### Evaluation 프로세스

1. 테스트 쿼리 생성
2. Agent 생성 및 쿼리 실행
3. 응답 수집 및 메트릭 측정
4. Evaluator로 품질 평가
5. 결과 저장 및 분석
6. Agent 정리

---

## 4. 테스트 쿼리 생성

Agent의 다양한 기능을 테스트할 쿼리를 생성합니다.

In [ ]:
# evals 디렉토리 생성
import os
from pathlib import Path

evals_dir = Path("evals")
evals_dir.mkdir(exist_ok=True)

print("=== Creating Evaluation Test Queries ===\n")

# 테스트 쿼리 정의
eval_queries = [
    {
        "query": "안녕하세요. 가족 여행지를 추천받고 싶어요.",
        "ground-truth": "Agent는 인사에 응답하고, 여행지 추천을 위해 Research Agent를 호출해야 합니다."
    },
    {
        "query": "부산의 현재 날씨를 알려주세요. 온도와 체감온도를 포함해주세요.",
        "ground-truth": "부산의 현재 날씨 정보를 정확하게 제공해야 하며, 온도와 체감온도를 모두 포함해야 합니다."
    },
    {
        "query": "제주도에서 가족과 함께 즐길 수 있는 여행지는 어디인가요?",
        "ground-truth": "제주도의 가족 친화적인 여행지를 검색하여 추천해야 합니다. 자연 명소, 체험 활동, 접근성 등을 고려한 정보를 제공합니다."
    },
    {
        "query": "서핑할 수 있는 해변을 추천해주세요.",
        "ground-truth": "서핑이 가능한 한국의 해변 여행지를 검색하여 추천해야 합니다. 양양, 부산 등의 서핑 명소 정보를 제공합니다."
    },
    {
        "query": "가을에 가기 좋은 단풍 명소는 어디인가요?",
        "ground-truth": "가을 계절에 방문하기 좋은 단풍 명소를 검색하여 추천해야 합니다. 내장산, 설악산 등의 자연 명소 정보를 제공합니다."
    }
]

# JSON 파일로 저장
eval_queries_path = evals_dir / "eval-queries.json"
with open(eval_queries_path, 'w', encoding='utf-8') as f:
    json.dump(eval_queries, f, indent=2, ensure_ascii=False)

print(f"✅ Created {eval_queries_path}")
print(f"\n📋 Test Queries ({len(eval_queries)} total):\n")

for i, query in enumerate(eval_queries, 1):
    print(f"   {i}. {query['query'][:60]}...")

print("\n💡 각 쿼리는 Agent의 다른 기능을 테스트합니다:")
print("   • 일반 대화 및 여행 의도 파악")
print("   • 날씨 조회 (Tool 기능)")
print("   • 여행지 지식 검색 (RAG 기능)")

print("\n" + "="*60)

## 5. Agent Evaluation 실행

평가용 Agent를 생성하고 테스트 쿼리를 실행한 후, Evaluator로 품질을 평가합니다.

### 참고사항

- **실행 시간**: 약 2-3분 소요
- **리전 제약**: eastus 리전에서는 일부 Safety Evaluator가 지원되지 않습니다
- **권한 이슈**: Azure AI Project의 스토리지 권한이 필요할 수 있습니다

In [ ]:
# Agent Evaluation 실행
import time
from pathlib import Path
from urllib.parse import urlparse
from azure.ai.agents.models import RunStatus, MessageRole
from azure.ai.projects import AIProjectClient
from azure.ai.evaluation import (
    AIAgentConverter, evaluate, ToolCallAccuracyEvaluator, IntentResolutionEvaluator, 
    TaskAdherenceEvaluator
)
from azure.identity import DefaultAzureCredential

print("=== Running Agent Evaluation ===\n")

# 파일 경로 설정
current_dir = Path(".")
evals_dir = current_dir / "evals"
eval_queries_path = evals_dir / "eval-queries.json"
eval_input_path = evals_dir / "eval-input.jsonl"
eval_output_path = evals_dir / "eval-output.json"

# 환경 변수 로드 (이미 config에서 로드됨)
project_endpoint = simple_project_conn
parsed_endpoint = urlparse(project_endpoint)
model_endpoint = f"{parsed_endpoint.scheme}://{parsed_endpoint.netloc}"
deployment_name = config.get("model_deployment_name", "gpt-4o")

print(f"📋 Configuration:")
print(f"   Project: {project_endpoint}")
print(f"   Model: {deployment_name}")
print(f"   Test Queries: {eval_queries_path}")
print(f"\n")

# Initialize AIProjectClient
print("🔌 Connecting to AI Project...")
credential = DefaultAzureCredential()
ai_project = AIProjectClient(
    credential=credential,
    endpoint=project_endpoint,
    api_version="2025-05-15-preview"  # Evaluations require preview API
)
print("✅ Connected\n")

# Evaluation용 Agent 생성
print("🤖 Creating Evaluation Agent...")
eval_agent = ai_project.agents.create_agent(
    model=deployment_name,
    name="Evaluation Agent",
    instructions="""You are a helpful travel and weather assistant.
    
You can help users with:
1. Travel recommendations and destination information
2. Weather information for any city
3. General travel planning advice

Be friendly, informative, and provide detailed responses."""
)
print(f"✅ Agent created: {eval_agent.name} (ID: {eval_agent.id})\n")

# Setup evaluation config
api_version = config.get("api_version", "2024-08-01-preview")
model_config = {
    "azure_deployment": deployment_name,
    "azure_endpoint": model_endpoint,
    "api_version": api_version,
}

thread_data_converter = AIAgentConverter(ai_project)

# 테스트 쿼리 실행 및 evaluation input 준비
print("="*70)
print("📝 Executing Test Queries\n")

with open(eval_queries_path, "r", encoding="utf-8") as f:
    test_data = json.load(f)

print(f"   Total queries: {len(test_data)}\n")

with open(eval_input_path, "w", encoding="utf-8") as f:
    for idx, row in enumerate(test_data, 1):
        query_text = row.get("query")
        print(f"   [{idx}/{len(test_data)}] {query_text[:50]}...")
        
        # 새 스레드 생성 (각 쿼리를 격리)
        thread = ai_project.agents.threads.create()
        
        # 사용자 쿼리 생성
        ai_project.agents.messages.create(
            thread.id, role=MessageRole.USER, content=query_text
        )
        
        # Agent 실행 및 성능 측정
        start_time = time.time()
        run = ai_project.agents.runs.create_and_process(
            thread_id=thread.id, agent_id=eval_agent.id
        )
        end_time = time.time()
        
        if run.status != RunStatus.COMPLETED:
            print(f"      ⚠️  Run failed: {run.last_error or 'Unknown error'}")
            continue
        
        # 운영 메트릭 수집
        operational_metrics = {
            "server-run-duration-in-seconds": (
                run.completed_at - run.created_at
            ).total_seconds(),
            "client-run-duration-in-seconds": end_time - start_time,
            "completion-tokens": run.usage.completion_tokens,
            "prompt-tokens": run.usage.prompt_tokens,
            "ground-truth": row.get("ground-truth", '')
        }
        
        # Thread 데이터 + 운영 메트릭을 evaluation input에 추가
        evaluation_data = thread_data_converter.prepare_evaluation_data(thread_ids=thread.id)
        eval_item = evaluation_data[0]
        eval_item["metrics"] = operational_metrics
        f.write(json.dumps(eval_item, ensure_ascii=False) + "\n")
        
        print(f"      ✅ Completed in {operational_metrics['client-run-duration-in-seconds']:.1f}s")
        print(f"         Tokens: {operational_metrics['prompt-tokens']} prompt + {operational_metrics['completion-tokens']} completion\n")

print("="*70)
print("\n✅ All queries executed successfully")
print(f"   Input saved to: {eval_input_path}\n")

# Evaluation 실행
print("="*70)
print("🔬 Running Evaluators\n")

print("   Evaluators:")
print("      • ToolCallAccuracyEvaluator")
print("      • IntentResolutionEvaluator")
print("      • TaskAdherenceEvaluator")
print("\n   ⚠️  참고: eastus 리전에서는 일부 RAI 평가자가 지원되지 않습니다.")
print("      (CodeVulnerability, ContentSafety, IndirectAttack는 제외)\n")
print("   ⏳ This may take 1-2 minutes...\n")

# OperationalMetricsEvaluator 정의
class OperationalMetricsEvaluator:
    """Propagate operational metrics to the final evaluation results"""
    def __init__(self):
        pass
    def __call__(self, *, metrics: dict, **kwargs):
        return metrics

# Evaluation 실행 (로컬)
print("   💡 평가 결과를 로컬에 저장합니다.\n")

results = evaluate(
    evaluation_name="foundry-agent-evaluation",
    data=eval_input_path,
    evaluators={
        "operational_metrics": OperationalMetricsEvaluator(),
        "tool_call_accuracy": ToolCallAccuracyEvaluator(model_config=model_config),
        "intent_resolution": IntentResolutionEvaluator(model_config=model_config),
        "task_adherence": TaskAdherenceEvaluator(model_config=model_config),
        # eastus 리전에서 지원되지 않는 평가자들은 제외
        # "code_vulnerability": CodeVulnerabilityEvaluator(credential=credential, azure_ai_project=project_endpoint),
        # "content_safety": ContentSafetyEvaluator(credential=credential, azure_ai_project=project_endpoint),
        # "indirect_attack": IndirectAttackEvaluator(credential=credential, azure_ai_project=project_endpoint)
    },
    output_path=eval_output_path,
    # azure_ai_project 파라미터 제거 (ML workspace 불필요)
)

print("✅ Evaluation completed!\n")
print(f"📁 결과 저장 위치: {eval_output_path}")
print(f"   다음 셀에서 결과를 확인하세요.\n")
print("="*70)

# Evaluation Agent 삭제
print("🧹 Cleaning up...")
ai_project.agents.delete_agent(eval_agent.id)
print(f"✅ Evaluation Agent deleted: {eval_agent.id}\n")

print("="*70)

## 6. Evaluation 결과 시각화

결과를 표와 차트로 시각화하여 분석합니다.

In [ ]:
# Evaluation 결과 시각화
import pandas as pd
import json
from pathlib import Path

print("=== 📊 Evaluation Results (평가 결과) ===\n")

# 파일에서 결과 로드
eval_output_path = Path("evals/eval-output.json")

if not eval_output_path.exists():
    print("❌ 평가 결과 파일을 찾을 수 없습니다.")
    print(f"   파일 경로: {eval_output_path}")
    print("\n   먼저 Cell 5를 실행하여 평가를 수행해주세요.\n")
else:
    with open(eval_output_path, 'r', encoding='utf-8') as f:
        results = json.load(f)
    
    metrics = results.get('metrics', {})
    rows = results.get('rows', [])
    
    # 1. 전체 점수 요약
    print("\n" + "="*80)
    print("⭐ OVERALL PERFORMANCE SCORES (전체 성능 점수)")
    print("="*80 + "\n")
    
    performance_scores = {}
    for key, value in metrics.items():
        if any(x in key.lower() for x in ['accuracy', 'resolution', 'adherence']) and isinstance(value, (int, float)):
            # 점수 이름 정리
            score_name = key.replace('.', ' ').replace('_', ' ').title()
            performance_scores[score_name] = value
    
    if performance_scores:
        for score_name, score in sorted(performance_scores.items(), key=lambda x: x[1], reverse=True):
            # 1-5 점수를 별표와 프로그레스 바로 표시
            stars = '★' * int(score) + '☆' * (5 - int(score))
            bar_length = int(score * 10)
            bar = '█' * bar_length + '░' * (50 - bar_length)
            print(f"{score_name:.<40} {score:.2f}/5.0  {stars}")
            print(f"{'':.<40} {bar}\n")
    else:
        print("   (성능 점수 데이터 없음)\n")
    
    # 2. 운영 메트릭 요약
    print("\n" + "="*80)
    print("⚡ OPERATIONAL METRICS (운영 메트릭)")
    print("="*80 + "\n")
    
    operational_data = {}
    for key, value in metrics.items():
        if 'duration' in key.lower():
            operational_data[f"⏱️  {key.replace('_', ' ').replace('-', ' ').title()}"] = f"{value:.2f}s"
        elif 'token' in key.lower():
            operational_data[f"🔤 {key.replace('_', ' ').replace('-', ' ').title()}"] = f"{int(value):,}"
    
    if operational_data:
        for metric_name, value in operational_data.items():
            print(f"   {metric_name:.<65} {value:>10}")
    else:
        print("   (운영 메트릭 데이터 없음)")
    
    # 3. 개별 쿼리 결과 상세 테이블
    print("\n\n" + "="*120)
    print("📋 DETAILED QUERY RESULTS (쿼리별 상세 결과)")
    print("="*120 + "\n")
    
    if rows:
        df = pd.DataFrame(rows)
        
        # 주요 컬럼 선택 및 정리
        display_data = []
        for idx, row in enumerate(rows, 1):
            query_text = row.get('query', '')[:45] + '...' if len(row.get('query', '')) > 45 else row.get('query', '')
            
            result_row = {
                'No.': idx,
                'Query': query_text,
                'Intent': row.get('intent_resolution.intent_resolution', '-'),
                'Task': row.get('task_adherence.task_adherence', '-'),
                'Tool': row.get('tool_call_accuracy.tool_call_accuracy', '-'),
                'Duration(s)': f"{row.get('client-run-duration-in-seconds', 0):.1f}",
                'Tokens': f"{row.get('prompt-tokens', 0)}+{row.get('completion-tokens', 0)}"
            }
            display_data.append(result_row)
        
        result_df = pd.DataFrame(display_data)
        print(result_df.to_string(index=False))
        
        # 평균 점수 계산
        print("\n" + "-"*120)
        intent_scores = [r.get('intent_resolution.intent_resolution') for r in rows if isinstance(r.get('intent_resolution.intent_resolution'), (int, float))]
        task_scores = [r.get('task_adherence.task_adherence') for r in rows if isinstance(r.get('task_adherence.task_adherence'), (int, float))]
        tool_scores = [r.get('tool_call_accuracy.tool_call_accuracy') for r in rows if isinstance(r.get('tool_call_accuracy.tool_call_accuracy'), (int, float))]
        
        if intent_scores:
            print(f"\n📈 Average Scores:")
            if intent_scores: print(f"   Intent Resolution:  {sum(intent_scores)/len(intent_scores):.2f}/5.0")
            if task_scores: print(f"   Task Adherence:     {sum(task_scores)/len(task_scores):.2f}/5.0")
            if tool_scores: print(f"   Tool Call Accuracy: {sum(tool_scores)/len(tool_scores):.2f}/5.0")
    else:
        print("   (쿼리 결과 없음)")
    
    print("\n" + "="*120)
    
    # 4. 점수 분포 차트
    print("\n\n" + "="*80)
    print("📊 SCORE DISTRIBUTION (점수 분포)")
    print("="*80 + "\n")
    
    if rows:
        # 각 메트릭별 점수 분포 계산
        for metric_name, scores in [
            ('Intent Resolution', intent_scores),
            ('Task Adherence', task_scores),
            ('Tool Call Accuracy', tool_scores)
        ]:
            if scores:
                avg_score = sum(scores) / len(scores)
                print(f"\n{metric_name}:")
                
                # 히스토그램
                for score in range(5, 0, -1):
                    count = sum(1 for s in scores if int(s) == score)
                    bar = '█' * count + '░' * (len(scores) - count)
                    print(f"   {score} {'★' * score:5}  [{count:2}]  {bar}")
                
                print(f"   {'─' * 40}")
                print(f"   Average: {avg_score:.2f}/5.0")
    
    print("\n" + "="*80)
    
    # 5. 파일 저장 위치
    print(f"\n\n✅ 평가 완료!")
    print(f"\n📁 상세 결과 파일:")
    print(f"   • {eval_output_path}")
    print(f"   • evals/eval-input.jsonl")
    print(f"\n💡 Tip: 위 파일을 열어서 더 자세한 내용을 확인할 수 있습니다.\n")

print("="*80)

## 7. Evaluation 메트릭 해석 가이드

### 메트릭 해석

**Operational Metrics:**
- `server-run-duration-in-seconds`: 서버에서 Agent 실행 시간
- `client-run-duration-in-seconds`: 클라이언트에서 측정한 총 소요 시간
- `prompt-tokens`: 입력 토큰 수
- `completion-tokens`: 생성된 토큰 수

**Performance Metrics:**
- `tool_call_accuracy.*`: 도구 호출 정확도 (1-5점)
- `intent_resolution.*`: 의도 파악 정확도 (1-5점)
- `task_adherence.*`: 작업 준수도 (1-5점)

**점수 해석:**
- 5점: 완벽함
- 4점: 좋음
- 3점: 보통
- 2점: 개선 필요
- 1점: 매우 나쁨

---

## 8. 다음 단계 및 Agent 개선

### Agent 개선 방법

1. **낮은 점수 분석**
   - 어떤 쿼리에서 점수가 낮았는지 확인
   - 어떤 evaluator에서 문제가 발생했는지 파악

2. **프롬프트 개선**
   - Agent instructions를 더 명확하게 작성
   - 예시 추가
   - 제약사항 명시

3. **기능 개선**
   - Tool 추가 또는 개선
   - RAG 지식 베이스 보강
   - Multi-Agent 조정

4. **재평가**
   - 동일한 쿼리로 재평가
   - 점수 변화 비교
   - 지속적인 개선

### 참고 자료

- [Azure AI Foundry Agent Evaluation](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/agent-evaluate-sdk)
- [Built-in Evaluators](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/evaluate-sdk)
- [Evaluation Best Practices](https://learn.microsoft.com/azure/ai-foundry/concepts/evaluation-approach-gen-ai)

---

## 완료!

축하합니다! Agent Evaluation을 성공적으로 완료했습니다. 🎉